# 2. Creating and Evaluating Solutions 

To recap from the first notebook:

1. Selecting a dataset.
1. Preparing interactions data for Personalize.
1. Creating a Dataset Group.
1. Creating and importing data into an Interactions dataset within the dataset group.


## Creating Solutions

This nobeook will cover creating the following solutions:

1. HRNN
1. SIMS
1. Personalized-Ranking

After that the metrics will be explained and another notebook will showcase how to interact with the Solutions once they are deployed into a Campaign.

The first step is to reload the imports and the stored variables from the previous notebooks.

In [1]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from datetime import datetime
import uuid

In [2]:
%store -r

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In Amazon Personalize, a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

We are going to list all the recipes that are supported. A recipe is an algorithm that has not been trained on your data yet. After listing, you'll select one and use that to build your model.

In [4]:
list_recipes_response = personalize.list_recipes()
print(list_recipes_response)

{'recipes': [{'name': 'aws-hrnn', 'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn', 'status': 'ACTIVE', 'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()), 'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 65000, tzinfo=tzlocal())}, {'name': 'aws-hrnn-coldstart', 'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart', 'status': 'ACTIVE', 'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()), 'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())}, {'name': 'aws-hrnn-metadata', 'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata', 'status': 'ACTIVE', 'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()), 'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())}, {'name': 'aws-personalized-ranking', 'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking', 'status': 'ACTIVE', 'creationDateTime': datetime.date

That is just a JSON representation of all of the algorithms that we have mentioned already. 

Next, we will select a particular algorithm then build a model with it.

----

### HRNN

HRNN is one of the more advanced recommendation models that you can use:
- It allows for things like real-time updates of recommendations based on user behavior. 
- It also tends to outperform other approaches like collaborative filtering. 
- We will kick this job off first as it takes the longest to complete.

#### Select Recipe

In [5]:
HRNN_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

#### Create and Wait for Solution

First, you will create the solution with the API, then you will create a version. 

Note the solution is just a label kind of identifier, you'll also need to create a version which is the actual trained model.

In [6]:
hrnn_create_solution_response = personalize.create_solution(
    name = "personalize-poc-hrnn"+str(uuid.uuid4())[:5],
    datasetGroupArn = dataset_group_arn,
    recipeArn = HRNN_recipe_arn
)

hrnn_solution_arn = hrnn_create_solution_response['solutionArn']
print(json.dumps(hrnn_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-hrnn011f5",
  "ResponseMetadata": {
    "RequestId": "19d6c1d8-605c-4301-9eb5-955cb8d266a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 06:40:48 GMT",
      "x-amzn-requestid": "19d6c1d8-605c-4301-9eb5-955cb8d266a4",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on.

In [7]:
hrnn_create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

In [8]:
hrnn_solution_version_arn = hrnn_create_solution_version_response['solutionVersionArn']
print(json.dumps(hrnn_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-hrnn011f5/4c7d5188",
  "ResponseMetadata": {
    "RequestId": "b8b557b0-b440-4c39-bfd5-ea69c6b591db",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 06:40:48 GMT",
      "x-amzn-requestid": "b8b557b0-b440-4c39-bfd5-ea69c6b591db",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


----

### SIMS


SIMS is one of the longest-running algorithms within Amazon for recommendation systems:
- A core use case for it is when you have one item, and you want to recommend items that have been interacted with in similar ways over your entire user base(not personalized). 
- Sometimes this leads to recommending mostly popular items, so there is a hyperparameter that can be tweaked that will reduce the popular items in your results. 

Just as last time we start by selecting the recipe:

#### Select Recipe

In [9]:
## SIMS:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### Create and Wait for Solution

As with HRNN, start with the solution first.

Note the solution is just a label kind of identifier, you'll also need to create a version which is the actual trained model.

In [10]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims"+str(uuid.uuid4())[:5],
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-sims9dc02",
  "ResponseMetadata": {
    "RequestId": "15efc3e5-04d7-4fd3-9538-680e13792e76",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 06:40:48 GMT",
      "x-amzn-requestid": "15efc3e5-04d7-4fd3-9538-680e13792e76",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on. 

In [11]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [12]:
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-sims9dc02/54de3bda",
  "ResponseMetadata": {
    "RequestId": "b99489d8-6f5b-4d7f-a92b-b351701fca60",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 06:40:48 GMT",
      "x-amzn-requestid": "b99489d8-6f5b-4d7f-a92b-b351701fca60",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalized Ranking

Personalized Ranking is an exciting application of HRNN:
- Instead of just recommending what is most probable for your user in question, this algorithm takes in a user and a list of items as well.
- The items are then rendered back in the order of most probability for the user. 
- For example, for filtering on the genre, or when you have a broad collection that you would like better ordered for a particular user.


#### Select Recipe

In [13]:
#### Personalized-rerank
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Create and Wait for Solution
First, you will create the solution with the API, then you will create a version. 

In [14]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank"+str(uuid.uuid4())[:5],
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-rerank871dc",
  "ResponseMetadata": {
    "RequestId": "d4b57b1b-3f55-4653-8a24-33bfedf87bae",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 06:40:49 GMT",
      "x-amzn-requestid": "d4b57b1b-3f55-4653-8a24-33bfedf87bae",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on.

In [15]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [16]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-rerank871dc/084fef94",
  "ResponseMetadata": {
    "RequestId": "2f18a2b1-79c1-4d8f-820a-ab70c2a08637",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 06:40:48 GMT",
      "x-amzn-requestid": "2f18a2b1-79c1-4d8f-820a-ab70c2a08637",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Viewing Solution Creation Status

As promised, how to view the status updates in the console:

* Go to [Personalize Console](https://console.aws.amazon.com/personalize/home?region=us-east-1#datasetGroups)
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the title.
* Click `Solutions and recipes.`
* You will now see a list of all of the solutions you created above. Click any one of them. 
* Note in `Solution versions` the job that is in progress. Once it is `Active`, your solution is ready to be reviewed. It is also capable of being deployed.



In [22]:
import threading

def threading_target(name, solution_version_arn):
    while True: 
        solutions_version_status = personalize.describe_solution_version(
            solutionVersionArn=solution_version_arn
        )["solutionVersion"]["status"]
        print(name, solutions_version_status)
        if solutions_version_status != 'ACTIVE' and solutions_version_status != 'CREATE_FAILED':
            time.sleep(30)
        else:
            break
        
solution_version_arns = {
    "rerank": rerank_solution_version_arn,
    "SIMS": sims_solution_version_arn,
    "HRNN": hrnn_solution_version_arn
}        
threads = list()
        
for key, solution_version_arn in solution_version_arns.items():
    thread = threading.Thread(target=threading_target, args=(key, solution_version_arn))
    threads.append(thread)
    thread.start()
    
for thread in threads: 
    thread.join()
    
print("All threads finished")    

HRNN ACTIVE
SIMS ACTIVE
rerank ACTIVE
All threads finished


## Evaluating Solutions

After about an hour max, the solutions should be ready for review. While they are in progress, it is a good idea to cover the various algorithms and their behavior in depth. You'll have another lull period as the solutions are being deployed into campaigns as well, so you can split the material into 2 sections if that makes it more accessible. Also, it can be an excellent time to discuss alternatives to how the data was fed into the system and what kind of results to expect from it.

The first step is to obtain the solutions' metrics, API calls for each below.

### HRNN Metrics:

In [23]:
hrnn_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(hrnn_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-hrnn011f5/4c7d5188",
  "metrics": {
    "coverage": 0.037,
    "mean_reciprocal_rank_at_25": 0.0284,
    "normalized_discounted_cumulative_gain_at_10": 0.0416,
    "normalized_discounted_cumulative_gain_at_25": 0.06,
    "normalized_discounted_cumulative_gain_at_5": 0.0346,
    "precision_at_10": 0.0077,
    "precision_at_25": 0.0062,
    "precision_at_5": 0.011
  },
  "ResponseMetadata": {
    "RequestId": "dc502d23-6d59-47b4-b3de-3942f02f8be9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 09:35:45 GMT",
      "x-amzn-requestid": "dc502d23-6d59-47b4-b3de-3942f02f8be9",
      "content-length": "405",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Metrics

Let's look at precision_at_K metrics, which depicts the number of relevant recommendations out of the top K recommendations divided by K. It gives about 1.1% for top 5 items.

Other metrics are available in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)

### Bias

Something else to note, the users' history is influenced by ANY recommendation model that is in place as your historical data is being collected: 
- This often means that while your model probably won't be this bad with a customer or on your own data, it does bias the metrics to favor their existing solution. 
- If you work to push the offline metrics to match or exceed their current solution, you may just be making HRNN start to behave like whatever they were already using.

### A/B

This is a great time to have a conversation about AB testing and to think about the actual business outcomes they are looking to drive:
- From there, you seem to run small experiments with Personalize against their existing recommendation system and see over time how the AB test performs. 
- If Personalize is winning, then it is the time to filter more and more traffic to Personalize and campaigns within it. 
- Over time the bias from the existing model will fade.

### SIMS Metrics:

In [24]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-sims9dc02/54de3bda",
  "metrics": {
    "coverage": 0.0889,
    "mean_reciprocal_rank_at_25": 0.0408,
    "normalized_discounted_cumulative_gain_at_10": 0.0425,
    "normalized_discounted_cumulative_gain_at_25": 0.0506,
    "normalized_discounted_cumulative_gain_at_5": 0.0297,
    "precision_at_10": 0.0141,
    "precision_at_25": 0.0092,
    "precision_at_5": 0.0129
  },
  "ResponseMetadata": {
    "RequestId": "22473750-4af4-47d3-880f-5897b46611f9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 09:35:50 GMT",
      "x-amzn-requestid": "22473750-4af4-47d3-880f-5897b46611f9",
      "content-length": "409",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In this example, we are seeing a slightly down-scaled precision at 5, a little over 0.7% this time:
- Effectively this is probably within the margin of error but given that no effort was made to mask popularity.
- It may just be returning super popular results that a large volume of users have interacted with in some way. 

### Personalized Ranking Metrics:

In [25]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:822894322603:solution/personalize-poc-rerank871dc/084fef94",
  "metrics": {
    "coverage": 0.0021,
    "mean_reciprocal_rank_at_25": 0.0398,
    "normalized_discounted_cumulative_gain_at_10": 0.0536,
    "normalized_discounted_cumulative_gain_at_25": 0.0627,
    "normalized_discounted_cumulative_gain_at_5": 0.04,
    "precision_at_10": 0.0091,
    "precision_at_25": 0.0057,
    "precision_at_5": 0.0103
  },
  "ResponseMetadata": {
    "RequestId": "e246447b-a0d8-4a44-acf3-8f67bd50af20",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 09:35:53 GMT",
      "x-amzn-requestid": "e246447b-a0d8-4a44-acf3-8f67bd50af20",
      "content-length": "409",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Because it is based on HRNN, the outcome has to be similar.

### Storing Useful Variables

Before exiting this notebook, run the following cells to save off our version arns for use later.

In [26]:
%store hrnn_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn

Stored 'hrnn_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'rerank_solution_version_arn' (str)


Just a quick comment on this one, here we see again a precision of near 1%, as this is based on HRNN, that is to be expected. 

You are now ready to move on to deploying and interacting with campaigns, continue by opening `Deploying_Campaigns_and_Interacting.ipynb`.